# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [20]:
spaceship = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv')
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [21]:
# drop empty values
initial_row_count = len(spaceship)
spaceship = spaceship.dropna()
final_row_count = len(spaceship)
dropped_rows = initial_row_count - final_row_count
# print(f'Dropped rows: {dropped_rows}\n{spaceship}')

# 'Deck' column to binaries
# extract the deck as the first letter, then create binary features for each deck
spaceship['Deck'] = spaceship['Cabin'].str.extract(r'([A-GT])', expand=False)

# create binary columns for each deck
spaceship = pd.get_dummies(spaceship, columns=['Deck'], prefix='Deck')

# drop columns
spaceship = spaceship.drop(columns=['PassengerId', 'Name'])

spaceship.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,False,True,False,False,False,False,False,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,False,False,False,False,False,True,False,False
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,False,False,False,False,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,False,False,False,False,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,False,False,False,False,False,True,False,False


In [22]:
# dummies from categorical data
spaceship_dummies = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP'])

spaceship_dummies.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck_A,Deck_B,Deck_C,...,Cabin_G/998/S,Cabin_G/999/P,Cabin_G/999/S,Cabin_T/1/P,Cabin_T/3/P,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,False,True,False,...,False,False,False,False,False,False,False,True,True,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,False,False,False,...,False,False,False,False,False,False,False,True,True,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,...,False,False,False,False,False,False,False,True,False,True
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,...,False,False,False,False,False,False,False,True,True,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,False,False,False,...,False,False,False,False,False,False,False,True,True,False


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

### Random Forest Classifier

In [23]:
# train test split

# separate features (X) and target (y)
X = spaceship_dummies.drop(columns=['Transported'])  # drop 'Transported' column as it's the target
y = spaceship_dummies['Transported']  # define target column

# train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# display resulting shapes
print(f'Training Features Shape: {X_train.shape}')
print(f'Testing Features Shape: {X_test.shape}')
print(f'Training Target Shape: {y_train.shape}')
print(f'Testing Target Shape: {y_test.shape}')

Training Features Shape: (5284, 5329)
Testing Features Shape: (1322, 5329)
Training Target Shape: (5284,)
Testing Target Shape: (1322,)


In [ ]:
# init
random_forest_model = RandomForestClassifier(
    n_estimators=10,    # Number of trees
    max_samples=0.8,     # Fraction of data each tree sees
    random_state=42
)

# train model
random_forest_model.fit(X_train, y_train)

# predictions
y_pred_rf = random_forest_model.predict(X_test)

- Evaluate your model

In [ ]:
accuracy = accuracy_score(y_test, y_pred_rf)
print(f'Accuracy: {accuracy:.4f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred_rf))

# Print confusion mtrix
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_pred_rf))

Accuracy: 0.7874

Classification Report:
              precision    recall  f1-score   support

       False       0.76      0.84      0.80       653
        True       0.83      0.73      0.78       669

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322


Confusion Matrix:
[[550 103]
 [178 491]]


In [26]:
cv_scores = cross_val_score(random_forest_model, X_train, y_train, cv=5)  # 5-fold cross-validation
print(f'\nCross-Validation Scores: {cv_scores}')
print(f'Mean Cross-Validation Score: {cv_scores.mean():.4f}')



Cross-Validation Scores: [0.7833491  0.75685904 0.7653737  0.76064333 0.7717803 ]
Mean Cross-Validation Score: 0.7676


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [ ]:
random_forest_model = RandomForestClassifier(random_state=42)

# hyperparameters to search over
param_grid = {
    'n_estimators': [10, 50, 100],               
    'max_depth': [None, 10],                      
    'min_samples_split': [2, 5],                   
    'min_samples_leaf': [1, 2],                    
    'bootstrap': [True],                           
}

# Grid Search with 3-fold cross-validation
grid_search = GridSearchCV(estimator=random_forest_model,
                           param_grid=param_grid,
                           cv=3,                    
                           n_jobs=1,                
                           verbose=1,               
                           scoring='accuracy')      

# fit grid search
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=1,
             param_grid={'bootstrap': [True], 'max_depth': [None, 10],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 50, 100]},
             scoring='accuracy', verbose=1)

- Evaluate your model

In [ ]:
# best parameters and score from grid search
print(f'Best Parameters: {grid_search.best_params_}')
print(f'Best Accuracy: {grid_search.best_score_:.4f}')

Fitting 3 folds for each of 24 candidates, totalling 72 fits
